In [1]:
# ! pip install -r requirements-dev.txt

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np

## Собираем табличку с трейн данными

In [3]:
transactions_df = pd.read_parquet("data/transactions.parquet").drop_duplicates()

In [38]:
# Размечаем транзакции:
# target=1 снятие наличных
# target=0 транзакции через терминал
target_df = pd.read_parquet("data/target.parquet").drop_duplicates()
target_df["target"] = 1
transactions_df = pd.merge(transactions_df, target_df, how="left", on = ["h3_09", "customer_id"])
transactions_df["target"] = transactions_df["target"].fillna(0).astype(int)

In [105]:
transactions_df["target"].value_counts().reset_index()

,target,count
0,0,3842534
1,1,308562


In [102]:
transactions_df

,h3_09,customer_id,datetime_id,count,sum,avg,min,max,std,count_distinct,mcc_code,target
0,8911aa4c62fffff,1,3,1,3346.65,3346.650,3346.65,3346.65,NaN,1,13,0
1,8911aa7b5b3ffff,4,3,1,450.00,450.000,450.00,450.00,NaN,1,8,0
2,8911aa63623ffff,5,3,10,11035.69,1103.569,59.00,3620.18,1190.530333,6,13,0
3,8911aa48577ffff,9,2,2,628.00,314.000,295.00,333.00,26.870058,2,5,0
4,8911aa78297ffff,11,2,1,4155.00,4155.000,4155.00,4155.00,NaN,1,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4151091,8911aa6216bffff,50600,2,1,4820.00,4820.000,4820.00,4820.00,NaN,1,0,0
4151092,891181b6423ffff,23168,1,1,70.00,70.000,70.00,70.00,NaN,1,13,0
4151093,8911aa7ab83ffff,62835,1,1,400.00,400.000,400.00,400.00,NaN,1,8,0
4151094,8911aa01dc7ffff,71229,1,1,6797.00,6797.000,6797.00,6797.00,NaN,1,5,0


## Посмотрим на выбранного клиента

In [110]:
customer_id = 23172
transactions_df[(transactions_df["customer_id"]==customer_id) & (transactions_df["target"]==1)]
# Чел в разные дни снимает в одном и том же банкомате (только однажды решил снять в 8911aa78c93ffff)

,h3_09,customer_id,datetime_id,count,sum,avg,min,max,std,count_distinct,mcc_code,target
319381,8911aa6ac3bffff,23172,1,1,3197.00,3197.000000,3197.00,3197.00,NaN,1,10,1
635329,8911aa6ac3bffff,23172,3,1,750.78,750.780000,750.78,750.78,NaN,1,5,1
743989,8911aa6ac3bffff,23172,3,1,600.00,600.000000,600.00,600.00,NaN,1,13,1
801294,8911aa6ac3bffff,23172,1,1,49.49,49.490000,49.49,49.49,NaN,1,13,1
833758,8911aa6ac3bffff,23172,1,1,260.00,260.000000,260.00,260.00,NaN,1,8,1
1190422,8911aa6ac3bffff,23172,2,7,4275.38,610.768571,36.99,1969.74,783.187378,4,13,1
1467110,8911aa6ac3bffff,23172,2,1,520.00,520.000000,520.00,520.00,NaN,1,8,1
1680488,8911aa6ac3bffff,23172,1,3,1710.00,570.000000,180.00,1090.00,468.721666,2,8,1
2109210,8911aa78c93ffff,23172,2,1,390.00,390.000000,390.00,390.00,NaN,1,8,1
2145851,8911aa6ac3bffff,23172,3,1,708.76,708.760000,708.76,708.76,NaN,1,8,1


In [111]:
transactions_df[(transactions_df["customer_id"]==customer_id) & (transactions_df["target"]==0)]
# Также у чела много транзакций в разных местах

,h3_09,customer_id,datetime_id,count,sum,avg,min,max,std,count_distinct,mcc_code,target
17375,8911aa79d57ffff,23172,2,1,23600.00,23600.000000,23600.00,23600.00,NaN,1,11,0
91371,8911aa72b5bffff,23172,3,1,50.00,50.000000,50.00,50.00,NaN,1,8,0
136789,8911aa44a27ffff,23172,2,1,399.97,399.970000,399.97,399.97,NaN,1,8,0
156267,8911aa6a4b7ffff,23172,2,1,1999.00,1999.000000,1999.00,1999.00,NaN,1,1,0
179028,8911aa7ac6fffff,23172,2,3,4900.00,1633.333333,900.00,3000.00,1184.62371,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3923467,8911aa6a4b7ffff,23172,3,1,999.00,999.000000,999.00,999.00,NaN,1,10,0
3935250,8911aa7ac63ffff,23172,2,1,100.00,100.000000,100.00,100.00,NaN,1,20,0
3948243,8911aa7a837ffff,23172,3,1,233.87,233.870000,233.87,233.87,NaN,1,13,0
3962777,8911aa6ad1bffff,23172,2,1,593.77,593.770000,593.77,593.77,NaN,1,13,0


## Еще файлы

In [67]:
# список всех 1658 геолокаций, где возможно снятие наличности, нужно разметить эти локации в решении
# тут 77 локаций, которых нет в transactions_df.query("target==1").h3_09.unique()
file_path = 'data/hexses_target.lst'
with open(file_path, 'r') as file:
    file_contents = file.read()
hexses_target = file_contents.split("\n")

In [114]:
# бесполезный файл, список всех уникальных h3_09 из transactions_df (из всего 8154)
file_path = 'data/hexses_data.lst'
with open(file_path, 'r') as file:
    file_contents = file.read()
hexses_data = file_contents.split("\n")

In [74]:
moscow_df = pd.read_parquet("data/moscow.parquet")

In [128]:
moscow_df.head()

,id,tags,lat,lon
0,1000,"[(comment, ООО ""АСП-ГРУПП"", ИНН 7735108290, до...",55.995115,37.207532
1,26609007,"[(ele, 190), (iata, SVO), (icao, UUEE), (name,...",55.987408,37.420944
2,26999673,"[(alt_name, Москва-Октябрьская), (esr:user, 06...",55.778834,37.653721
3,26999677,None,55.784881,37.649458
4,26999678,None,55.785265,37.649081


In [125]:
for tag in moscow_df[~moscow_df.tags.isna()].tags.iloc[:20]:
    print(tag)

[('comment', 'ООО "АСП-ГРУПП", ИНН 7735108290, до 22:45 c 2014-05-23'), ('name', 'Десяточка'), ('opening_hours', '08:30-22:45'), ('shop', 'supermarket')]
[('ele', '190'), ('iata', 'SVO'), ('icao', 'UUEE'), ('name', 'Шереметьево'), ('source', 'wikipedia'), ('website', 'http://www.sheremetyevo-airport.ru'), ('wikidata', 'Q192733'), ('wikipedia', 'ru:Шереметьево'), ('wikipedia:en', 'Sheremetyevo_International_Airport')]
[('alt_name', 'Москва-Октябрьская'), ('esr:user', '060073'), ('loc_name', 'Москва-Ленинградская'), ('loc_name:website', 'http://www.tutu.ru/station.php?nnst=79310'), ('name', 'Москва-Пассажирская'), ('nat_name', 'Москва'), ('official_name', 'Москва-Пассажирская'), ('official_name:esr', 'Москва-Пассажирская'), ('official_name:esr:website', 'http://cargo.rzd.ru/cargostation/public/ru?STRUCTURE_ID=5101&layer_id=4829&page4821_2705=1&refererLayerId=4821&id=1090'), ('official_name:express-3', 'Москва-Октябрьская'), ('official_name:express-3:website', 'http://bilet.ru/rus/TrainDi

In [ ]:
fig = px.scatter_mapbox(moscow_df.iloc[:10000], lat='lat', lon='lon', hover_name='id', zoom=10, hover_data=["tags"])
fig.update_layout(mapbox_style="open-street-map", height=1000)
fig.update_layout(title='Plot of Points', hovermode='closest')
fig.show()